In [1]:
#using the UFRC python kernel
#general libs
import pandas as pd
import numpy as np
import subprocess
import time
import glob
import os
import sys
import config

#biopython
from Bio import PDB
from Bio.SVDSuperimposer import SVDSuperimposer

#!pip install py3Dmol
import py3Dmol


print('library import success')

library import success


In [2]:
config.config


{'DIRECTORY': {'INPUT': '/blue/kgraim/zfg2013/ExoSearch/Input',
  'ROOT': '/blue/kgraim/zfg2013/ExoSearch',
  'HDOCK': '/blue/kgraim/zfg2013/ExoSearch/Input/HDOCK',
  'OUTPUT': '/blue/kgraim/zfg2013/ExoSearch/Output'},
 'SLURM': {'account': 'kgraim', 'qos': 'kgraim', 'job': 'ES', 'tasks': 5}}

In [3]:
#data import, for a list of fastas and their target receptors in a csv as a colabfold multimer formatted fasta file. 
#Otherwise, this block will find fasta files in the directory

#Change over to the input directory
if os.getcwd() != config.config['DIRECTORY']['INPUT']:
    os.chdir(config.config['DIRECTORY']['INPUT'])

#finding file types
def file_AF_finder():
    try:
        data_fasta = glob.glob("*.fasta")
        data_csv = glob.glob("*.csv")

        if data_fasta:
            print("FASTA input found")
            print("fasta format selected")
            print("Files are :", data_fasta)
            return data_fasta
        elif data_csv:
            print("CSV input found")
            print("CSV format selected, fasta files were unavailable")
            print("Files are :", data_csv)
            return data_csv
        else:
            print("No matching data types found")

        print("FASTA not found")
        print("CSV not found")

    except Exception as e:
        print("Error occurred while checking data types:", str(e))
     
    

def AF(file):
    file_type = str(file)
    try:
        if 'fasta'.lower() in file_type.lower():
            print('Fasta file confirmed, moving on with AlphaFold')
            os.chdir(config.config['DIRECTORY']['ROOT'])
            process = subprocess.Popen(['sbatch', array_flag, 'colab_fasta.sh'])
            
        elif 'csv'.lower() in file_type.lower():
            print('CSV file confirmed, moving on with AlphaFold')
            process = subprocess.Popen(['sbatch', 'colab_csv.sh'])
            os.chdir(config.config['DIRECTORY']['ROOT'])
            
        else:
            print('No matching file type found')
            return
        
        process.wait()
        
        if process.returncode == 0:
            print("AlphaFold initiated")
        else:
            print("Error occurred while executing AlphaFold")
            
    except Exception as e:
        print("Error occurred:", str(e))       
        
#
af_data = file_AF_finder() 
array_flag = "--array=1-{}".format(str(len(af_data)))
AF(af_data)



FASTA input found
fasta format selected
Files are : ['gRNA-1_d.fasta', 'tropomyosin_kinase.fasta', 'shaker-1.fasta', 'gRNA_b.fasta']
Fasta file confirmed, moving on with AlphaFold
Submitted batch job 14721825
AlphaFold initiated


In [10]:
!squeue -u zfg2013

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          14721627       gpu sys/dash  zfg2013  R    1:23:04      1 c0904a-s11


In [15]:
# this block will find the files to use HDOCK Lite v1.1 - 
#HDOCK requires pdbs for the receptor and ligands, make sure they're matched by name 
#e.g receptor_1; ligand_1

#finding file types
def file_HD_finder():
    
    if os.getcwd() != config.config['DIRECTORY']['HDOCK']:
        os.chdir(config.config['DIRECTORY']['HDOCK'])
    
    try:
        data_receptor = sorted(glob.glob("*receptor*.pdb"))
        data_ligand = sorted(glob.glob("*ligand*.pdb"))

        if data_receptor:
            print("Receptor input found")
            receptors = [receptor for sublist in data_receptor for receptor in sublist.split(",")]
           
        else:
            print("Receptor not found")

        if data_ligand:
            print("Ligand input found")
            ligands = [ligand for sublist in data_ligand for ligand in sublist.split(",")]
        
        else:
            print("Ligand not found")
        
        # Combine into DataFrame
        data = {'Receptor': receptors, 'Ligand': ligands}
        hdock_frame = pd.DataFrame(data)
        print("matched fasta file #: ", len(hdock_frame))
        print("please inspect to confirm matched files exist \n", hdock_frame)
        return hdock_frame
                                   
    except Exception as e:
        print("Error occurred while checking data types:", str(e))

#docking
def HDOCK(file):
    if os.getcwd() != config.config['DIRECTORY']['HDOCK']:
        os.chdir(config.config['DIRECTORY']['HDOCK'])             
    try:
        process = subprocess.Popen(['sbatch', array_flag, 'HDOCK.sh'])
        process.wait()
        if process.returncode == 0:
            print("Docking initiated")
        else:
            print("Error occurred while executing HDOCK")
            
    except Exception as e:
        print("Error occurred:", str(e))
        

#the output here is model(N).pdb        
hdock_data = file_HD_finder()
array_flag = "--array=1-{}".format(str(len(hdock_data)))                 
                 
#prepare to do the docking 
HDOCK(hdock_data)

Receptor input found
Ligand input found
matched fasta file #:  1
please inspect to confirm matched files exist 
          Receptor        Ligand
0  receptor_1.pdb  ligand_1.pdb
Submitted batch job 14010048
Docking initiated


In [1]:
#Testing block

amber_config ='''#!/bin/sh
#SBATCH --account=kgraim
#SBATCH --qos=kgraim
#SBATCH --job-name=Jupyter
#SBATCH --ntasks=5
#SBATCH --ntasks-per-socket=1
#SBATCH --threads-per-core=1
#SBATCH --nodes=1
#SBATCH --distribution=cyclic:cyclic
#SBATCH --mem-per-cpu=6000mb
#SBATCH --time=4:00:00 
pwd; hostname; date

#load modules
module purge
module load intel/2017.4.056 amber/16
export PATH=$HPC_AMBER_DIR/AmberTools/bin:$PATH

#go to input directory
input="/blue/kgraim/zfg2013/ExoSearch/Amber/Input/AF_AMBER" 

#base structure
complex=$(ls $input)

#process by amber
pdb4amber -i "$input"/$complex -o "$input"/p_"$complex" --prot --reduce --dry

#split your protein complex into the separate proteinsg 

cat <<EOF >> "$input"/chain_split.R
library("bio3d")
library("XML")
setwd("$input")
pdbsplit('p_$complex', path="/blue/kgraim/zfg2013/ExoSearch/Amber/Input/AF_AMBER/")
EOF



cat <<EOF >> "$input"/chain_split.sh
#!/bin/sh
#go to input directory
input="/blue/kgraim/zfg2013/ExoSearch/Amber/Input/AF_AMBER" 
ml R/4.1
Rscript "$input"/chain_split.R
EOF


#Create amber preprocessing file through leap
base=$(basename p_"$complex" .pdb)

cat <<EOF >> "$input"/leap.in

input="/blue/kgraim/zfg2013/ExoSearch/Amber/Input/AF_AMBER" 
source leaprc.protein.ff14SB
source leaprc.water.tip3p
source leaprc.gaff2
protein_complex = loadPdb $input/$base.pdb
A = loadPdB $input/${base}_A.pdb
B = loadPdB $input/${base}_B.pdb
saveamberparm protein_complex $input/c.prmtop $input/c.inpcrd
saveamberparm A $input/A.prmtop $input/A.inpcrd
saveamberparm B $input/B.prmtop $input/B.inpcrd
charge protein_complex
solvatebox protein_complex TIP3PBOX 12.0
saveamberparm protein_complex $input/cs.prmtop $input/cs.inpcrd
quit
EOF

source "$input"/chain_split.sh
tleap -f "$input"/leap.in 



cat <<EOF >> "$input"/min.in
minimise the protein complex
 &cntrl
  imin=1,maxcyc=1000,ncyc=500,
  cut=8.0,ntb=1,
  ntc=2,ntf=2,
  ntpr=100,
  ntr=1, restraintmask=':1-242',
  restraint_wt=2.0
 /
EOF

cat <<EOF >>  "$input"/heat.in
heat the protein complex
&cntrl
  imin=0,irest=0,ntx=1,
  nstlim=25000,dt=0.002,
  ntc=2,ntf=2,
  cut=8.0, ntb=1,
  ntpr=500, ntwx=500,
  ntt=3, gamma_ln=2.0,
  tempi=0.0, temp0=300.0, ig=-1,
  ntr=1, restraintmask=':1-242',
  restraint_wt=2.0,
  nmropt=1
 /
 &wt TYPE='TEMP0', istep1=0, istep2=25000,
  value1=0.1, value2=300.0, /
 &wt TYPE='END' /
EOF


cat <<EOF >> "$input"/density.in
heat stage part 2
&cntrl
  imin=0,irest=0,ntx=1,
  nstlim=25000,dt=0.002,
  ntc=2,ntf=2,
  cut=8.0, ntb=1,
  ntpr=500, ntwx=500,
  ntt=3, gamma_ln=2.0,
  tempi=0.0, temp0=300.0, ig=-1,
  ntr=1, restraintmask=':1-242',
  restraint_wt=2.0,
  nmropt=1
 /
 &wt TYPE='TEMP0', istep1=0, istep2=25000,
  value1=0.1, value2=300.0, /
 &wt TYPE='END' /
EOF

cat <<EOF >> "$input"/equil.in
finally equilibriated
&cntrl
  imin=0,irest=0,ntx=1,
  nstlim=25000,dt=0.002,
  ntc=2,ntf=2,
  cut=8.0, ntb=1,
  ntpr=500, ntwx=500,
  ntt=3, gamma_ln=2.0,
  tempi=0.0, temp0=300.0, ig=-1,
  ntr=1, restraintmask=':1-242',
  restraint_wt=2.0,
  nmropt=1
 /
 &wt TYPE='TEMP0', istep1=0, istep2=25000,
  value1=0.1, value2=300.0, /
 &wt TYPE='END' /
EOF

#Part 1
sander -O -i "$input"/min.in -o "$input"/min.out -p "$input"/cs.prmtop -c "$input"/cs.inpcrd -r "$input"/min.rst -ref "$input"/cs.inpcrd

#Step 2
sander -O -i "$input"/heat.in -o "$input"/heat.out -p "$input"/cs.prmtop -c "$input"/min.rst  -r "$input"/heat.rst -x "$input"/heat.mdcrd -ref "$input"/min.rst
gzip -9 "$input"/heat.mdcrd

#Step 3
sander -O -i "$input"/density.in -o "$input"/density.out -p "$input"/cs.prmtop -c "$input"/heat.rst -r "$input"/density.rst -x "$input"/density.mdcrd -ref "$input"/heat.rst
gzip -9 "$input"/density.mdcrd

#Step 4
sander -O -i "$input"/equil.in -o "$input"/equil.out -p "$input"/cs.prmtop -c "$input"/density.rst -r "$input"/equil.rst -x "$input"/equil.mdcrd
gzip -9 "$input"/equil.mdcrd

exit 0
'''


#write the bash file
with open("amber.sh" ,"w+") as f:
    f.writelines(amber_config)
    f.close()

!sbatch amber.sh - #alphafold 

Submitted batch job 13090058


In [79]:
#Once all the files are generated, now we do the alignment. Let's assume we pick the top outputs from each residing
#in the output folder. Will change to scalable implements later

def aligner():
    
    if os.getcwd() != config.config['DIRECTORY']['OUTPUT']:
        os.chdir('~/ExoSearch/Output')
        
    try:
        AF = sorted(glob.glob("AF*.pdb"))
        AFS = [model for sublist in AF for model in sublist.split(",")]
        
        HDOCK = sorted(glob.glob("HDOCK*.pdb"))
        HDOCKS = [model for sublist in HDOCK for model in sublist.split(",")]
        
        data = {'AF': AFS, 'HDOCK': HDOCKS}
        df = pd.DataFrame(data)
        
        if len(df) < (len(AFS) or len(HDOCKS)):  # Cause we have hdock and af available
            print("Missing matched file")
            if 'AF' not in str(AFS):
                print("AF data missing")
                
            if 'HDOCK' not in str(HDOCKS):
                print("HDOCK data missing")
        else:
            print("Matched files found, performing the alignment")

            # Preparing for the alignment
            parser = PDB.PDBParser(QUIET=True)
            AF_model = parser.get_structure("AF", outputs[0])
            HDOCK_model = parser.get_structure("HDOCK", outputs[1])
            

            # Get the atoms and coordinates for just the receptor chain
            atoms_AF = PDB.Selection.unfold_entities(AF_model[0]['A'], 'A')
            af_coordinates = np.array([atom.get_coord() for atom in atoms_AF])

            
            atoms_HDOCK = PDB.Selection.unfold_entities(HDOCK_model[0]['A'], 'A') #atom object 
            hdock_coordinates = np.array([atom.get_coord() for atom in atoms_HDOCK]) #atom coordinates 
 
            
            # Perform the alignment
            superimposer = SVDSuperimposer()
            superimposer.set(af_coordinates, hdock_coordinates) #hdock will be rotated and translated onto alphafold
            superimposer.run()
            hdock_on_af = superimposer.get_transformed() #these are the coordinates for hdock onto af, update the atom object with these coordinates
            
            #update ##add checkstep later
            for i in range(len(atoms_HDOCK)):
                for i in range(len(hdock_on_af)):
                    atoms_HDOCK[i].set_coord(hdock_on_af[i])
            
            #update teh coordinates
            io = PDB.PDBIO()
            io.set_structure(HDOCK_model)
            
            #now lets compare the RMSD 
            atoms_AFP, atoms_HDP = PDB.Selection.unfold_entities(AF_model[0]['B'], 'A'), PDB.Selection.unfold_entities(HDOCK_model[0]['B'],'A')
            afP_coordinates, hdp_coordinates  = np.array([atom.get_coord() for atom in atoms_AFP]), np.array([atom.get_coord() for atom in atoms_HDP])
            rmsd = np.sqrt(np.mean(np.sum((afP_coordinates - hdp_coordinates) ** 2, axis=1)))
            print("RMSD between peptides", rmsd)
        
            
            #write the new structure 
            #output_path = "Updated_hdock.pdb"
            #io.save(output_path)
            print("completed")
            
            return rmsd
            

                
    except Exception as e:
        print("Error occurred:", str(e))
        
 
    
aligner()

Matched files found, performing the alignment
Error occurred: name 'outputs' is not defined


In [267]:
#viewing the aligned structure (HDOCK aligned vs NON)
with open("Updated_hdock.pdb") as ifile:
    system="".join([x for x in ifile])

with open("HDOCK.pdb") as ifile:
    old_system="".join([x for x in ifile])
    
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.addModelsAsFrames(old_system)

# Set colors for chains A and B in the first model
view.setStyle({'model': 0, 'chain': 'A'}, {"cartoon": {'color': 'red'}})
view.setStyle({'model': 0, 'chain': 'B'}, {"cartoon": {'color': 'blue'}})

# Set colors for chains A and B in the second model
view.setStyle({'model': 1, 'chain': 'A'}, {"cartoon": {'color': 'black'}})
view.setStyle({'model': 1, 'chain': 'B'}, {"cartoon": {'color': 'green'}})

#add title label
view.addLabel("HDOCK receptor alignment to AF vs native HDOCK, RMSD: 0", {"position": {'x':-39.89, 'y':45.75, 'z':0.35},"fontSize":12})

#view
view.zoomTo()
view.show()

rmsd = 3.0503526

#viewing the aligned structure (ALIGNED HDOCK TO AF)
with open("Updated_hdock.pdb") as ifile:
    system="".join([x for x in ifile])

with open("AF.pdb") as ifile:
    af_system="".join([x for x in ifile])
    
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.addModelsAsFrames(af_system)

# Set colors for chains A and B in the first model #HDOCK
view.setStyle({'model': 0, 'chain': 'A'}, {"cartoon": {'color': 'red'}})
view.setStyle({'model': 0, 'chain': 'B'}, {"cartoon": {'color': 'blue'}})

# Set colors for chains A and B in the second model #ALPHAFOLD
view.setStyle({'model': 1, 'chain': 'A'}, {"cartoon": {'color': 'black'}})
view.setStyle({'model': 1, 'chain': 'B'}, {"cartoon": {'color': 'green'}})

#add title label
view.addLabel(f"HDOCK / Alphafold Peptide Comparison, RMSD:{rmsd}", {"position": {'x':-39.89, 'y':45.75, 'z':0}, "fontSize":12})

#view
view.zoomTo()
view.show()




You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol